In [1]:
import rawpy
import imageio
import torch
import matplotlib.pyplot as plt
import cv2
import numpy as np
import torch.nn.functional as F
from PIL import Image
import random
import os

In [2]:
def pack_avg_cst(dir_path,raw_filename):
    raw_filename = raw_filename.split(".")[0]
    current_path = dir_path + "/" + raw_filename + ".ARW"
    with rawpy.imread(current_path) as raw:
        # Get raw image data
        image = np.array(raw.raw_image, dtype=np.float16)
        # subtract black levels and normalize to interval [0..1]
        black = np.reshape(np.array(raw.black_level_per_channel, dtype=np.float16), (2, 2))
        black = np.tile(black, (image.shape[0]//2, image.shape[1]//2))
        image = ((image - black) / (raw.white_level - black))
        # Get the raw pattern of the photo
        n_colors = raw.num_colors
        colors = np.frombuffer(raw.color_desc, dtype=np.byte)
        pattern = np.array(raw.raw_pattern)
        i_0 = np.where(colors[pattern] == colors[0])
        i_1 = np.where(colors[pattern] == colors[1])
        i_2 = np.where(colors[pattern] == colors[2])

        # Pack image and average the green channels
        image_packed = np.empty((image.shape[0]//2, image.shape[1]//2, n_colors))
        
        image_packed[:, :, 0] = image[i_0[0][0]::2, i_0[1][0]::2]
        image_packed[:, :, 1]  = (image[i_1[0][0]::2, i_1[1][0]::2] + image[i_1[0][1]::2, i_1[1][1]::2]) / 2
        image_packed[:, :, 2]  = image[i_2[0][0]::2, i_2[1][0]::2]

        # CST Conversion
        cst = raw.rgb_xyz_matrix[0:3, :]
        image_XYZ = np.einsum('ij,...j', cst, image_packed)
        image_XYZ = torch.from_numpy(image_XYZ)

        print(image_XYZ.shape)
        image_sRGB = np.clip(image_XYZ, 0, 1)
        # show image
        #plt.axis('off')
        #plt.imshow(image_sRGB)
        print(image_sRGB.shape)
        im = Image.fromarray((np.asarray(image_sRGB) * 255).astype(np.uint8))
        im.save('C:/Users/Revi/Desktop/Sony/preprocessed_pngs/'+raw_filename+".png")
        #return image_XYZ #Returns Tensor with size [1,H,W,C]


In [4]:
# List w/ all images in folder
dir_path = 'C:/Users/Revi/Desktop/Sony/sony_train/'
file_list = os.listdir(dir_path)
random.seed(42)
random.shuffle(file_list)


for current_image in file_list[2300:2320]:
    #current_image = file_list[208]
    current_path = dir_path + "/" + current_image
    prep_image = pack_avg_cst(dir_path,current_image)


torch.Size([1836, 2752, 3])
torch.Size([1836, 2752, 3])


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Revi/Desktop/Sony/preprocessed_pngs/DSC04122.png'

In [9]:
%reset -f